In [2]:
import numpy as np

def squeeze_bg(data, bg_value=0):
    """
    Crop 3D image to smallest bounding box containing all non-background voxels.
    
    Parameters:
    -----------
    data : numpy.ndarray
        3D image array from nii_img.get_fdata()
    bg_value : float or int
        Background value (default: 0)
    
    Returns:
    --------
    numpy.ndarray
        Cropped 3D image array
    """
    # Find indices where data is not background
    non_bg = np.where(data != bg_value)
    
    # If entire image is background, return as is
    if len(non_bg[0]) == 0:
        return data
    
    # Find bounding box coordinates
    min_x, max_x = non_bg[0].min(), non_bg[0].max()
    min_y, max_y = non_bg[1].min(), non_bg[1].max()
    min_z, max_z = non_bg[2].min(), non_bg[2].max()
    
    # Crop the image (add 1 to max indices to include them)
    cropped = data[min_x:max_x+1, min_y:max_y+1, min_z:max_z+1]
    
    return cropped

In [5]:
import numpy as np

def pad_crop_to_cube(data, n, pad_value=0):
    """
    Pad and crop 3D image to (n, n, n) dimension.
    Padding is applied on both sides, cropping removes from the back.
    
    Parameters:
    -----------
    data : numpy.ndarray
        3D image array
    n : int
        Target dimension size for all axes
    pad_value : float or int
        Value to use for padding (default: 0)
    
    Returns:
    --------
    numpy.ndarray
        Resized 3D image array of shape (n, n, n)
    """
    current_shape = data.shape
    
    # Process each axis independently
    for axis in range(3):
        current_size = data.shape[axis]
        
        if current_size < n:
            # Padding needed - pad on both sides
            total_pad = n - current_size
            pad_before = total_pad // 2
            pad_after = total_pad - pad_before
            
            # Create padding configuration
            pad_width = [(0, 0), (0, 0), (0, 0)]
            pad_width[axis] = (pad_before, pad_after)
            
            # Apply padding
            data = np.pad(data, pad_width, mode='constant', constant_values=pad_value)
            
        elif current_size > n:
            # Cropping needed - crop from the back
            if axis == 0:
                data = data[:n, :, :]
            elif axis == 1:
                data = data[:, :n, :]
            else:  # axis == 2
                data = data[:, :, :n]
    
    return data

In [7]:
import nibabel as nib
from pathlib import Path

input_folder = Path(r"D:\Kananat\Data\Last0\2_Masked")
output_folder = Path(r"D:\Kananat\Data\Last0\3_Preprocessed")

i = 0
for file in input_folder.glob("*.nii.gz"):
    i+=1
    print(f"{i} Processing file: {file.name}")

    file_path = str(file)
    nii_img = nib.load(file_path)

    data = nii_img.get_fdata()
    data = np.clip(data, -400, 1600)
    data = squeeze_bg(data, bg_value=-400)
    data = pad_crop_to_cube(data, n=256, pad_value=-400)
    data = (data + 400) / 2000  # Normalize to [0, 1]

    new_img = nib.Nifti1Image(data.astype(np.float32), nii_img.affine, nii_img.header)
    output_path = output_folder / file.name.replace("_masked.nii.gz", "_preprocessed.nii.gz")
    
    nib.save(new_img, output_path)
print("Done")

1 Processing file: 47-16872 L_masked.nii.gz
2 Processing file: 47-16872 R_masked.nii.gz
3 Processing file: 47-22136 L_masked.nii.gz
4 Processing file: 47-22136 R_masked.nii.gz
5 Processing file: 47-4881 L 2014_masked.nii.gz
6 Processing file: 47-4881 L 2018_masked.nii.gz
7 Processing file: 47-4881 R 2014_masked.nii.gz
8 Processing file: 47-4881 R 2018_masked.nii.gz
9 Processing file: 48-26453 L_masked.nii.gz
10 Processing file: 48-5955 L_masked.nii.gz
11 Processing file: 48-5955 R_masked.nii.gz
12 Processing file: 49-18165 L_masked.nii.gz
13 Processing file: 49-18165 R_masked.nii.gz
14 Processing file: 49-3614 L_masked.nii.gz
15 Processing file: 49-3614 R_masked.nii.gz
16 Processing file: 50-11620 L 20250213_masked.nii.gz
17 Processing file: 50-11620 R 20250213_masked.nii.gz
18 Processing file: 50-30909 R_masked.nii.gz
19 Processing file: 51-26987 L_masked.nii.gz
20 Processing file: 51-26987 R_masked.nii.gz
21 Processing file: 51-28114 L_masked.nii.gz
22 Processing file: 51-28114 R_mas